In [52]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import os, sys
from sklearn.metrics.pairwise import cosine_similarity



# get cosine similairty matrix
def cos_sim(input_vectors):
    similarity = cosine_similarity(input_vectors)
    return similarity

# get use score for a sentence
def get_use_score(prof_sentence, s,sentences_list, similarity_matrix):
    # find the index of sentence in list
    prof_index = sentences_list.index(prof_sentence)
    stud_index = sentences_list.index(s)
    # get the corresponding row in similarity matrix
    similarity_score= similarity_matrix[prof_index,stud_index]
    return similarity_score

# get score of N-Grams
def gram_score(teacher_sentence, student_sentence):
    teacher_list = teacher_sentence.split(" ")


module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [53]:
data = pd.DataFrame(np.zeros((30,3)),columns =['Students/Prof','Questions','Answer'])
data['Students/Prof'] = ['Prof','Stud1','Stud2','Stud3','Stud4','Stud5','Stud6','Stud7','Stud8','Stud9','Prof','Stud1','Stud2','Stud3','Stud4','Stud5','Stud6','Stud7','Prof','Stud1','Stud2','Stud3','Stud4','Stud5','Prof','Stud1','Stud2','Stud3','Stud4','Stud5']
data['Questions'] = ['What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?',\
                     'Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?',\
                     'PM of India?','PM of India?','PM of India?','PM of India?','PM of India?','PM of India?',\
                     'What is Data Science?','What is Data Science?','What is Data Science?','What is Data Science?','What is Data Science?','What is Data Science?']
data['Answer']=["Science is a branch of knowledge or study dealing. with a body of facts or truth systematically arranged and showing the operation of. general laws. It is systematic knowledge of the physical or material world gained. through observation and experimentation.",\
                "Science is the study of the nature and behaviour of natural things and the knowledge that we obtain about them. ... A science is a particular branch of science such as physics, chemistry, or biology. Physics is the best example of a science which has developed strong, abstract theories.",\
                "a branch of knowledge or study dealing with a body of facts or truths systematically arranged and showing the operation of general laws: the mathematical sciences. systematic knowledge of the physical or material world gained through observation and experimentation.",\
                "Knowledge about or study of the natural world based on facts learned through experiments and observation. : a particular area ofscientific study (such as biology, physics, or chemistry) : a particular branch of science",\
                "Science is a branch of knowledge or study dealing. with a body of facts or truth systematically arranged and showing the operation of. general laws. It is systematic knowledge of the physical or material world gained. through observation and experimentation.",\
                "The aim of science is to build true and accurate knowledge about how the world works. The word 'truth' is sometimes used to refer to spiritual truths or other topics that science cannot investigate. To be interested in scientific truth, one doesn't have to reject other sources of meaning.",\
                "knowledge about or study of the natural world based on facts learned through experiments and observation. : a particular area of scientific study (such as biology, physics, or chemistry) : a particular branch of science.",\
                "the intellectual and practical activity encompassing the systematic study of the structure and behaviour of the physical and natural world through observation and experiment.",\
                "Science is defined as the observation, identification, description, experimental investigation, and theoretical explanation of natural phenomena.",\
                "Science is known as research and study of all global phenomena ",\
                "Infront of Forum mall, 7th block koramangla, Bangalore",\
                "Koramangla, near forum mall",\
                "Koramangla, bangalore karnataka",\
                "close to forum mall, koramangla",\
                "far from forum mall, bangalore, koramangla",\
                "not near forum mall, bangalore, koramangla",\
                "not near white field, bangalore, koramangla",\
                "tavarekere road, bangalore, koramangla",\
                "Narendra Modi",\
                "N. Modi",\
                "Modi",\
                "Narendra damordas Modi",\
                "Amit Shah",\
                "Amit Modi",\
                "Data science is a 'concept to unify statistics, data analysis, machine learning and their related methods' in order to 'understand and analyze actual phenomena' with data. It employs techniques and theories drawn from many fields within the context of mathematics, statistics, information science, and computer science",\
                "Data science is a multi-disciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from data in various forms, both structured and unstructured,[1][2] similar to data mining",\
                "data science is the analysis of existing data to discover patterns",\
                "A data scientist, based on my current understanding, is the person who connects the dots between the business world and the data world.",\
                "Data Science is the study of where information comes from, what it represents and how it can be turned into a valuable resource in the creation of business and IT strategies",\
                "Data Science involves using automated methods to analyze massive amounts of data and to extract knowledge from them"]

In [54]:
tf.__version__

'1.15.0'

In [55]:
# Reduce logging output.
# tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings = session.run(embed(list(data['Answer'])))
    similarity_matrix = cos_sim(np.array(sentences_embeddings)) 

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [56]:
use_score_list=[]
for i in range(len(data)):
    question = data['Questions'][i]
    stud_ans = data['Answer'][i]
    prof_ans = list(data[(data['Questions']==question) & (data['Students/Prof']=='Prof')]['Answer'])[0]
    all_ans = list(data[(data['Questions']==question)]['Answer'])

    use_score = get_use_score(prof_ans, stud_ans, all_ans, similarity_matrix)
    use_score_list+=[use_score]
data['USE_score'] = use_score_list

In [61]:
score_list=[]
index_prof = np.where(np.array(list(data['Students/Prof']))=='Prof')[0]

for i,index in enumerate(index_prof):
    index_prof_ans = index
    prof_ans = data['Answer'].iloc[index]
    if i==len(index_prof)-1:
        index_student_ans = range(index+1,len(data))
    else:
        index_student_ans = range(index+1,index_prof[i+1])
    
    score_list.append(similarity_matrix[index,index])
    if len(prof_ans.split())<=3:
        for j in index_student_ans:
            intersection = set(prof_ans.split()).intersection(set(data['Answer'].iloc[j].split()))
            score_list.append(len(intersection)/len(prof_ans.split()))   
    else:
        for j in index_student_ans:
            score_list.append(similarity_matrix[index,j])

In [62]:
data['new_score'] = score_list
data

,Students/Prof,Questions,Answer,USE_score,new_score
0,Prof,What is the definition of science ?,Science is a branch of knowledge or study deal...,1.000000,1.000000
1,Stud1,What is the definition of science ?,Science is the study of the nature and behavio...,0.827074,0.827074
2,Stud2,What is the definition of science ?,a branch of knowledge or study dealing with a ...,0.947231,0.947231
3,Stud3,What is the definition of science ?,Knowledge about or study of the natural world ...,0.779670,0.779670
4,Stud4,What is the definition of science ?,Science is a branch of knowledge or study deal...,1.000000,1.000000
5,Stud5,What is the definition of science ?,The aim of science is to build true and accura...,0.716361,0.716361
6,Stud6,What is the definition of science ?,knowledge about or study of the natural world ...,0.785190,0.785190
7,Stud7,What is the definition of science ?,the intellectual and practical activity encomp...,0.826412,0.826412
8,Stud8,What is the definition of science ?,"Science is defined as the observation, identif...",0.799785,0.799785
9,Stud9,What is the definition of science ?,Science is known as research and study of all ...,0.713160,0.713160


In [65]:
#question classification
import re
def get_dataframe(filename):
    lines = open(filename, 'r').read().splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i].split(' ')[0]
        label = label.split(":")[0]
        text = ' '.join(lines[i].split(' ')[1:])
        text = re.sub('[^A-Za-z0-9 ,\?\'\"-._\+\!/\`@=;:]+', '', text)
        data.append([label, text])

    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

In [80]:
df_train = get_dataframe('train.txt')
df_test = get_dataframe('test.txt')
df_train.head()

,label,text
0,DESC,How did serfdom develop in and then leave Russ...
1,ENTY,What films featured the character Popeye Doyle ?
2,DESC,How can I find a list of celebrities ' real na...
3,ENTY,What fowl grabs the spotlight after the Chines...
4,ABBR,What is the full form of .com ?


In [81]:
train_text = df_train['text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

train_label = np.asarray(pd.get_dummies(df_train.label), dtype = np.int8)

test_text = df_test['text'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

test_label = np.asarray(pd.get_dummies(df_test.label), dtype = np.int8)

In [68]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), 
    	signature="default", as_dict=True)["default"]

In [75]:
from keras import layers
from keras import Model
embed_size = 512
category_counts = 6
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding,
	output_shape=(embed_size,))(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(category_counts, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', 
	optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [82]:
# from keras import K
with tf.Session() as session:
#   K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=10,
            batch_size=32)
  model.save_weights('./model.h5')

Train on 5452 samples, validate on 500 samples
Epoch 1/10
5452/5452 [==============================] - 40s 7ms/step - loss: 0.7197 - accuracy: 0.8067 - val_loss: 0.3260 - val_accuracy: 0.8900
Epoch 2/10
5452/5452 [==============================] - 33s 6ms/step - loss: 0.3248 - accuracy: 0.8916 - val_loss: 0.2433 - val_accuracy: 0.9300
Epoch 3/10
5452/5452 [==============================] - 32s 6ms/step - loss: 0.2833 - accuracy: 0.9054 - val_loss: 0.2253 - val_accuracy: 0.9260
Epoch 4/10
5452/5452 [==============================] - 32s 6ms/step - loss: 0.2603 - accuracy: 0.9110 - val_loss: 0.2251 - val_accuracy: 0.9280
Epoch 5/10
5452/5452 [==============================] - 32s 6ms/step - loss: 0.2457 - accuracy: 0.9142 - val_loss: 0.2074 - val_accuracy: 0.9380
Epoch 6/10
5452/5452 [==============================] - 32s 6ms/step - loss: 0.2362 - accuracy: 0.9187 - val_loss: 0.2380 - val_accuracy: 0.9280
Epoch 7/10
5452/5452 [==============================] - 32s 6ms/step - loss: 0.2242

In [84]:
new_text = ["In what year did the titanic sink ?", 
            "What is the highest peak in California ?", 
            "Who invented the light bulb ?"]

new_text =  ['What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?','What is the definition of science ?',\
                     'Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?','Where is RBEI location?',\
                     'PM of India?','PM of India?','PM of India?','PM of India?','PM of India?','PM of India?',\
                     'What is Data Science?','What is Data Science?','What is Data Science?','What is Data Science?','What is Data Science?','What is Data Science?']

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
#   K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text, batch_size=32)

categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
print(predict_labels)

['DESC', 'DESC', 'DESC', 'DESC', 'DESC', 'DESC', 'DESC', 'DESC', 'DESC', 'DESC', 'LOC', 'LOC', 'LOC', 'LOC', 'LOC', 'LOC', 'LOC', 'LOC', 'HUM', 'HUM', 'HUM', 'HUM', 'HUM', 'HUM', 'DESC', 'DESC', 'DESC', 'DESC', 'DESC', 'DESC']
